In [189]:
with open('libraries.py') as f:
    code = f.read()
exec(code)

with open('functions.py') as f:
    code = f.read()
exec(code)

In [190]:
pd.reset_option('all')
pd.set_option('display.max_rows', 100)

In [191]:
# determine user
user = getpass.getuser()
if user == 'peymansh':
    main_folder_path = '/Users/peymansh/Dropbox (MIT)/Research/AI and Occupations/ai-exposure'
    data_path = f'{main_folder_path}/output'

In [192]:
onet_data_path = f'{data_path}/data/onet_occupations_yearly.csv'

# list of occupations to create DAGs for
occupation_list = ['travelAgents', 'insuranceUnderwriters', 'pileDriverOperators', 
                   'dredgeOperators', 'gradersAndSortersForAgriculturalProducts', 'reinforcingIronAndRebarWorkers',
                   'insuranceAppraisersForAutoDamage', 'floorSandersAndFinishers', 'dataEntryKeyer', 
                   'athletesAndSportsCompetitors', 'audiovisualEquipmentInstallerAndRepairers', 'hearingAidSpecialists', 
                   'personalCareAides', 'proofreadersAndCopyMarkers', 'chiropractors', 
                   'shippingReceivingAndInventoryClerks', 'cooksShortOrder', 'orthodontists',
                   'subwayAndStreetcarOperators', 'packersAndPackagersHand', 'hoistAndWinchOperators', 
                   'forgingMachineSettersOperatorsAndTenders', 'avionicsTechnicians', 'dishwashers', 
                   'dispatchersExceptPoliceFireAndAmbulance', 'familyMedicinePhysicians', 'MachineFeedersAndOffbearers'
                   ]

occupation = 'travelAgents'
# occupation = 'insuranceUnderwriters'
# occupation = 'pileDriverOperators'
# occupation = 'athletesAndSportsCompetitors'
occupation = 'audiovisualEquipmentInstallerAndRepairers'

# Generate occupation-specific strings
GPT_input_occupation, plot_title_occupation, occupation_code, occupation_folder = pick_occupation(occupation)

In [193]:
# set alpha as AI quality metric
n = 100
epsilon = 1e-8
alpha_list = np.linspace(epsilon, 1-epsilon, n).tolist()

### Initialize input-output paths

In [194]:
# Manual DAG
input_path = f'{occupation_folder}/{occupation}_M_DAG_df.csv'
output_path = f'{occupation_folder}/{occupation}_costMin_M.csv'

# Naive DAG
input_path = f'{occupation_folder}/{occupation}_N_GPT_DAG_df.csv'
output_path = f'{occupation_folder}/{occupation}_costMin_N.csv'

# Conditioned Naive DAG
input_path = f'{occupation_folder}/{occupation}_CN_GPT_DAG_df.csv'
output_path = f'{occupation_folder}/{occupation}_costMin_CN.csv'

# # First Last Task DAG
# input_path = f'{occupation_folder}/{occupation}_FLT_GPT_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_FLT.csv'

# # Conditioned First Last Task DAG
# input_path = f'{occupation_folder}/{occupation}_CFLT_GPT_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_CFLT.csv'

# # Partitioned DAG
# input_path = f'{occupation_folder}/{occupation}_P_GPT_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_P.csv'

# # Conditioned Partitioned DAG
# input_path = f'{occupation_folder}/{occupation}_CP_GPT_DAG_df.csv'
# output_path = f'{occupation_folder}/{occupation}_costMin_CP.csv'

In [195]:
# read DAG
dag_df = pd.read_csv(input_path)

# remove edges if comment column labeled with "TriangleRemovedFlag" (edge is there for plotting purposes and is not part of the actual DAG)
if 'comment' in dag_df.columns:
    dag_df = dag_df[~dag_df['comment'].str.endswith('TriangleRemovedFlag')]

# get task stats
tasks_stats = pd.read_csv(f'{occupation_folder}/{occupation}_taskStats.csv')
tasks_stats

# print stats
#tasks_stats.iloc[:,1:].sum()
dag_df

,source,target,comment
2,"Install, service, and repair electronic equipm...","Calibrate and test equipment, and locate circu...",The worker calibrating and testing equipment n...
7,"Calibrate and test equipment, and locate circu...",Compute cost estimates for labor and materials.,The worker computing cost estimates for labor ...
8,Disassemble entertainment equipment and repair...,"Install, service, and repair electronic equipm...","The worker installing, servicing, and repairin..."
10,Read and interpret electronic circuit diagrams...,Disassemble entertainment equipment and repair...,The worker disassembling and repairing equipme...
11,Confer with customers to determine the nature ...,Keep records of work orders and test and maint...,The worker keeping records of work orders and ...
12,"Calibrate and test equipment, and locate circu...",Keep records of work orders and test and maint...,The worker keeping records of work orders and ...
14,Tune or adjust equipment and instruments to ob...,"Position or mount speakers, and wire speakers ...",The worker positioning or mounting speakers an...
21,Compute cost estimates for labor and materials.,"Position or mount speakers, and wire speakers ...",The worker positioning or mounting speakers an...
22,Tune or adjust equipment and instruments to ob...,Keep records of work orders and test and maint...,The worker keeping records of work orders and ...
24,"Calibrate and test equipment, and locate circu...",Make service calls to repair units in customer...,The worker making service calls to repair unit...


In [196]:
# extract list of tasks and create a dictionary for indexing tasks
tasks_list = tasks_stats['task'].unique()
tasks_dict = {i: node for i, node in enumerate(tasks_list, start=0)}

# create numpy array of adjacency matrix
adjacency_matrix = np.zeros((len(tasks_list), len(tasks_list)), dtype=int)
aux_dict = {value: key for key, value in tasks_dict.items()}
for _, row in dag_df.iterrows():
    source_index = aux_dict[row['source']]
    target_index = aux_dict[row['target']]
    adjacency_matrix[source_index, target_index] = 1

tasks_dict

{0: 'Disassemble entertainment equipment and repair or replace loose, worn, or defective components and wiring, using hand tools and soldering irons.',
 1: 'Install, service, and repair electronic equipment or instruments such as televisions, radios, and videocassette recorders.',
 2: 'Calibrate and test equipment, and locate circuit and component faults, using hand and power tools and measuring and testing instruments such as resistance meters and oscilloscopes.',
 3: 'Confer with customers to determine the nature of problems or to explain repairs.',
 4: 'Tune or adjust equipment and instruments to obtain optimum visual or auditory reception, according to specifications, manuals, and drawings.',
 5: 'Instruct customers on the safe and proper use of equipment.',
 6: 'Compute cost estimates for labor and materials.',
 7: 'Read and interpret electronic circuit diagrams, function block diagrams, specifications, engineering drawings, and service manuals.',
 8: 'Keep records of work orders 

In [197]:
def find_neighbors(adjacency_matrix):
    # Get the number of nodes (n) from the shape of the adjacency matrix
    n = adjacency_matrix.shape[0]
    
    # Initialize an empty dictionary to store the neighbors for each node
    neighbors = {i: [] for i in range(n)}
    
    # Loop through each entry in the adjacency matrix
    for i in range(n):
        for j in range(n):
            # If there's an edge from i to j or from j to i, add j to the neighbors of i
            if adjacency_matrix[i, j] == 1 or adjacency_matrix[j, i] == 1:
                if j not in neighbors[i]:  # Avoid duplicate neighbors
                    neighbors[i].append(j)
                if i not in neighbors[j]:  # Ensure symmetry in the undirected version
                    neighbors[j].append(i)
    
    return neighbors

In [198]:
# def create_inactive_node_neighbor_subset_combinations(inactive_neighbors_valid_subsets_dict):
#     # Step 1: Extract lists from dictionary and remove duplicates within lists
#     all_lists = [list(set(item)) for sublist in inactive_neighbors_valid_subsets_dict.values() for item in sublist]
#     print(f'Number of lists extracted: {len(all_lists)}')

#     # Step 2: Create all combinations of lists across keys
#     output_set = set()
#     for r in range(1, len(all_lists) + 1):
#         combinations = itertools.combinations(all_lists, r)
#         for combo in combinations:
#             # Flatten the combination of lists
#             flattened_combo = list(itertools.chain(*combo))
#             # Remove duplicates within the flattened list and sort for consistency
#             unique_combo = tuple(sorted(set(flattened_combo)))
#             # Add the unique combination to the output set
#             output_set.add(unique_combo)

#     # Convert the set back to a list of lists
#     output_list = [list(combo) for combo in output_set]
#     return sorted(output_list, key=len)



def create_inactive_node_neighbor_subset_combinations(inactive_neighbors_valid_subsets_dict):
    # Step 1: Extract unique lists from the dictionary values
    all_lists = [list(set(item)) for sublist in inactive_neighbors_valid_subsets_dict.values() for item in sublist]
    print(f'Number of lists extracted: {len(all_lists)}')

    # Step 2: Create all combinations and directly add unique elements
    output_set = set()
    
    # Instead of recomputing length and duplicates, work with unique sets directly
    all_combinations = []
    
    for r in range(1, len(all_lists) + 1):
        for combo in itertools.combinations(all_lists, r):
            # Convert each combination to a flattened tuple of sorted unique elements
            flattened_combo = tuple(sorted(set(itertools.chain(*combo))))
            output_set.add(flattened_combo)  # Add to set to ensure uniqueness
    
    # Convert the set back to sorted list of lists and return the result
    output_list = [list(combo) for combo in output_set]
    
    return sorted(output_list, key=len)

In [199]:
def get_valid_DAG_subsets(adjacency_matrix):
    # subset adjacency matrix to exclude Target node
    non_target_adjacency_matrix = adjacency_matrix[:-1,:-1].copy()

    # get number of non-Target nodes
    n = non_target_adjacency_matrix.shape[0]


    # def valid_subsets_recursive(adjacency_matrix, global_active_dict, global_memory_dict, node):
    #     active_dict = global_active_dict.copy()
    #     memory_dict = global_memory_dict.copy()
    def valid_subsets_recursive(adjacency_matrix, active_dict, memory_dict, partition):
        print(f'\npartition {partition}')

        # if partition already in memory return its value
        try:
            if len(memory_dict[tuple(sorted(partition))]) > 0:
                print(f'partition {partition} already calculated:', memory_dict[tuple(sorted(partition))])
                return memory_dict[tuple(sorted(partition))]
            
        # if partition not in memory, get valid subsets of partition
        except KeyError:
            # get inactive neighbors of node
            partition_neighbors = find_neighbors(adjacency_matrix)
            print(f'neighbors of partition {partition}:', partition_neighbors)
            neighbors_list = []
            for node in partition:
                neighbors_list.append(partition_neighbors[node])
            
            # delete repetitions and keep unique neighbors
            neighbors_list = list(itertools.chain(*neighbors_list))
            neighbors_list = list(set(neighbors_list))
            print(f'neighbors of partition {partition}:', neighbors_list)

            inactive_neighbors_list = [neighbor for neighbor in neighbors_list if active_dict[neighbor] == False]
            print(f'inactive neighbors of partition {partition}:', inactive_neighbors_list)

            # if partition has no outgoing edges return partition and empty list
            if len(inactive_neighbors_list) == 0:
                print(f'partition {partition} is has no outgoing edges, return:', [partition, []])
                memory_dict[tuple(sorted(partition))] = [partition, []]
                return [partition, []]

            # for each inactive neighbor get valid subsets
            inactive_neighbors_valid_subsets = {}



            ############
            # set all neighbors to active and get valid subsets of neighbors
            for neighbor in inactive_neighbors_list:
                active_dict[neighbor] = True
            # print(f'\n\n>>>>>>>>>active dict: {active_dict}<<<<<<<<<')
            ############


            # create all subsets of inactive neighbors to loop over
            inactive_neighbor_subsets = []
            for r in range(len(inactive_neighbors_list) + 1):
                inactive_neighbor_subsets.extend(itertools.combinations(inactive_neighbors_list, r))
            inactive_neighbor_subsets = [list(subset) for subset in inactive_neighbor_subsets if len(subset) > 0]
            print(f'inactive neighbor subsets:', inactive_neighbor_subsets)


            for neighbor_partition in inactive_neighbor_subsets:
                print(f'\n-----------Running partition {partition}, neighbor partition {neighbor_partition}-----------')

                ############
                # # set neighbor to active and get valid subsets of neighbor
                # active_dict[neighbor] = True
                print(f'\n\n>>>>>>>>>active dict: {active_dict}<<<<<<<<<')
                ############


                ############
                # make a copy of active_dict and memory dict for the neighbor contingencies
                neighbor_active_dict = active_dict.copy()
                neighbor_memory_dict = memory_dict.copy()
                ############

                valid_subsets = valid_subsets_recursive(adjacency_matrix, neighbor_active_dict, neighbor_memory_dict, neighbor_partition)

                # add neighbor partition itself to valid subsets of neighbor
                #valid_subsets = [subset + neighbor_partition for subset in valid_subsets]
                valid_subsets = sorted(valid_subsets, key=len)
                print(f'valid subsets of neighbor partition {neighbor_partition}:', valid_subsets)

                # add valid subsets of neighbor to memory
                memory_dict[tuple(sorted(neighbor_partition))] = valid_subsets
                print(f'memory dict:', memory_dict)

                # append valid subsets of neighbor to valid subsets of node
                inactive_neighbors_valid_subsets[tuple(sorted(neighbor_partition))] = valid_subsets

            # create all combinations of valid subsets of inactive neighbors
            for key, lists in inactive_neighbors_valid_subsets.items():
                #lists.append([])
                for inner_list in lists:
                    inner_list.extend(partition)
            print(f'valid subsets of inactive neighbor partitoins of partition {partition}:', inactive_neighbors_valid_subsets)            
            
            inactive_neighbors_valid_subsets_combinations = []
            for key, value in inactive_neighbors_valid_subsets.items():
                print(key, value)
                inactive_neighbors_valid_subsets_combinations.extend(value)
            
            # add partition itself as a valid subset
            inactive_neighbors_valid_subsets_combinations.append(partition)

            # drop repetitions
            inactive_neighbors_valid_subsets_combinations = [sorted(list(subset)) for subset in set(tuple(subset) for subset in inactive_neighbors_valid_subsets_combinations)]
            
            # sort combinations by length
            inactive_neighbors_valid_subsets_combinations = sorted(inactive_neighbors_valid_subsets_combinations, key=len)

            # filter out empty subsets -- mainly for compatibility reasons due to last nodes (i.e., nodes w/o outgoing edges)
            inactive_neighbors_valid_subsets_combinations = [subset for subset in inactive_neighbors_valid_subsets_combinations if subset != []]
            print(f'combinations of valid subsets of inactive neighbors of partition {partition}:', inactive_neighbors_valid_subsets_combinations)

            return inactive_neighbors_valid_subsets_combinations
                

            


    # initialize dictionary for valid subsets origniating from each node
    valid_subsets_dict = {}
    
    # run the algorithm on each node and remove that node from the adjacency matrix after each iteration
    my_adjacency_matrix = non_target_adjacency_matrix.copy()
    for node in range(n):
        print(f'***********Running node {node}***********')
        print(f'adjacency matrix:\n', my_adjacency_matrix)

        # create active dictionary
        global_active_dict = {i: False for i in range(n)}

        # set node as active
        global_active_dict[0] = True

        # initialize dict for valid subsets of nodes (and also partitions) to act as memory
        global_memory_dict = {}

        # get valid subsets of node 0 in current adjacency matrix
        valid_subsets = valid_subsets_recursive(my_adjacency_matrix, global_active_dict, global_memory_dict, [0])
        print(f'\nmemory_dict:', global_memory_dict)

        # adjust output above for actual node number in main adjacency matrix
        valid_subsets_dict[node] = [[element + node for element in subset] for subset in valid_subsets]

        # ensure empty subsets do not exist in valid subsets
        valid_subsets_dict[node] = [subset for subset in valid_subsets_dict[node] if len(subset) > 0]
        print(f'\nvalid subsets of node {node}:', valid_subsets_dict[node], '\n\n\n')

        # remove current node from adjacency matrix for next iteration
        my_adjacency_matrix = my_adjacency_matrix[1:,1:]
    
    return valid_subsets_dict

In [200]:
# example_adjacency_matrix = np.array([[0, 1, 1, 1, 0],
#                                      [0, 0, 0, 1, 0],
#                                      [0, 0, 0, 1, 0],
#                                      [0, 0, 0, 0, 0],
#                                      [0, 0, 0, 0, 0]])

# valid_subsets_dict = get_valid_DAG_subsets(example_adjacency_matrix)
# valid_subsets_dict

In [201]:
# example_adjacency_matrix = np.array([[0, 0, 1, 0, 0],
#                                      [0, 0, 1, 0, 0],
#                                      [0, 0, 0, 1, 0],
#                                      [0, 0, 0, 0, 1],
#                                      [0, 0, 0, 0, 0]])


# valid_subsets_dict = get_valid_DAG_subsets(example_adjacency_matrix)
# valid_subsets_dict

In [202]:
example_adjacency_matrix = np.array([[0, 1, 1, 0, 0],
                                     [0, 0, 0, 1, 0],
                                     [0, 0, 0, 1, 0],
                                     [0, 0, 0, 0, 0],
                                     [0, 0, 0, 0, 0]])

valid_subsets_dict = get_valid_DAG_subsets(example_adjacency_matrix)
valid_subsets_dict

***********Running node 0***********
adjacency matrix:
 [[0 1 1 0]
 [0 0 0 1]
 [0 0 0 1]
 [0 0 0 0]]

partition [0]
neighbors of partition [0]: {0: [1, 2], 1: [0, 3], 2: [0, 3], 3: [1, 2]}
neighbors of partition [0]: [1, 2]
inactive neighbors of partition [0]: [1, 2]
inactive neighbor subsets: [[1], [2], [1, 2]]

-----------Running partition [0], neighbor partition [1]-----------


>>>>>>>>>active dict: {0: True, 1: True, 2: True, 3: False}<<<<<<<<<

partition [1]
neighbors of partition [1]: {0: [1, 2], 1: [0, 3], 2: [0, 3], 3: [1, 2]}
neighbors of partition [1]: [0, 3]
inactive neighbors of partition [1]: [3]
inactive neighbor subsets: [[3]]

-----------Running partition [1], neighbor partition [3]-----------


>>>>>>>>>active dict: {0: True, 1: True, 2: True, 3: True}<<<<<<<<<

partition [3]
neighbors of partition [3]: {0: [1, 2], 1: [0, 3], 2: [0, 3], 3: [1, 2]}
neighbors of partition [3]: [1, 2]
inactive neighbors of partition [3]: []
partition [3] is has no outgoing edges, return:

{0: [[0], [0, 2], [0, 1], [0, 1, 2], [0, 1, 3], [0, 2, 3], [0, 1, 2, 3]],
 1: [[1], [1, 3], [1, 2, 3]],
 2: [[2], [2, 3]],
 3: [[3]]}

In [203]:
valid_subsets_dict = get_valid_DAG_subsets(adjacency_matrix)
valid_subsets_dict

***********Running node 0***********
adjacency matrix:
 [[0 1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 1 0 1 1 0]
 [0 0 0 0 0 0 0 1 1 0 0]
 [0 0 0 0 0 1 0 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 1]
 [1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0]]

partition [0]
neighbors of partition [0]: {0: [1, 7], 1: [0, 2], 2: [1, 4, 6, 8, 9], 3: [7, 8], 4: [2, 5, 8, 9, 10], 5: [4], 6: [2, 9, 10], 7: [0, 3], 8: [2, 3, 4, 9], 9: [2, 4, 6, 8], 10: [4, 6]}
neighbors of partition [0]: [1, 7]
inactive neighbors of partition [0]: [1, 7]
inactive neighbor subsets: [[1], [7], [1, 7]]

-----------Running partition [0], neighbor partition [1]-----------


>>>>>>>>>active dict: {0: True, 1: True, 2: False, 3: False, 4: False, 5: False, 6: False, 7: True, 8: False, 9: False, 10: False}<<<<<<<<<

partition [1]
neighbors of partition [1]: {0: [1, 7], 1: [0, 2], 2: [1, 4, 6, 8, 9], 3: [7, 8], 4: [2, 5, 8, 9, 10], 5: [4], 6: [2, 9

{0: [[0],
  [0, 7],
  [0, 1],
  [0, 1, 2],
  [0, 3, 7],
  [0, 1, 7],
  [0, 1, 2, 4],
  [0, 1, 2, 8],
  [0, 1, 2, 6],
  [0, 1, 2, 7],
  [0, 1, 2, 9],
  [0, 3, 7, 8],
  [0, 1, 3, 7],
  [0, 1, 2, 3, 8],
  [0, 1, 2, 4, 10],
  [0, 1, 2, 6, 9],
  [0, 1, 2, 4, 7],
  [0, 1, 2, 4, 6],
  [0, 1, 2, 8, 9],
  [0, 1, 2, 4, 9],
  [0, 1, 2, 7, 8],
  [0, 1, 3, 7, 8],
  [0, 1, 2, 6, 7],
  [0, 1, 2, 4, 5],
  [0, 2, 3, 7, 8],
  [0, 1, 2, 3, 7],
  [0, 1, 2, 6, 8],
  [0, 3, 4, 7, 8],
  [0, 1, 2, 6, 10],
  [0, 1, 2, 4, 8],
  [0, 3, 7, 8, 9],
  [0, 1, 2, 7, 9],
  [0, 1, 2, 4, 5, 10],
  [0, 1, 2, 4, 8, 9],
  [0, 1, 2, 6, 7, 10],
  [0, 1, 2, 3, 7, 9],
  [0, 1, 2, 3, 4, 8],
  [0, 1, 2, 4, 6, 8],
  [0, 2, 3, 7, 8, 9],
  [0, 1, 2, 4, 9, 10],
  [0, 1, 2, 6, 8, 9],
  [0, 1, 2, 4, 7, 8],
  [0, 1, 2, 6, 7, 8],
  [0, 1, 2, 3, 6, 8],
  [0, 1, 2, 4, 7, 10],
  [0, 1, 2, 6, 9, 10],
  [0, 1, 2, 4, 5, 9],
  [0, 1, 2, 3, 8, 9],
  [0, 1, 3, 4, 7, 8],
  [0, 1, 2, 4, 8, 10],
  [0, 1, 2, 3, 7, 8],
  [0, 1, 2, 4, 6, 9],
  [0, 3, 4

In [204]:
def is_combination_valid(combination, n):
    # Flatten list of combination
    covered_tasks_list = [element for sublist in combination for element in sublist]
    
    # Create a set of the flattened list
    covered_tasks_set = set(covered_tasks_list)
    
    # Check if the flattened set has exactly n elements and contains all elements from 0 to n-1
    if len(covered_tasks_list) == n and covered_tasks_set == set(range(n)):
        return True
    else:
        return False


def generate_combinations(valid_subsets_dict, current_key=0, current_combination=None, result=None):
    if current_combination is None:
        current_combination = []
    if result is None:
        result = []

    # Base case: if convered all tasks add current combination to the result list
    if is_combination_valid(current_combination, len(valid_subsets_dict)):
        print(f'********found valid combination: {current_combination}********')
        result.append(current_combination)
        return result

    # Recursive case: iterate through the list of lists at the current key
    for subset in valid_subsets_dict[current_key]:
        # Create a new combination including the current subset
        new_combination = current_combination + [subset]
        new_combination_flattened = [element for sublist in new_combination for element in sublist]
        
        # lower the load of computation by skipping invalid combinations
        # combination is invalid if:
        # 1. length of new combination is greater than the length of valid subsets dictionary
        # 2. new combination contains repetitive elements

        # skip cases in which the new combination is invalid
        if len(new_combination_flattened) > len(valid_subsets_dict):
            continue
        if len(new_combination_flattened) != len(set(new_combination_flattened)):
            continue

        # print(f'new combination: {new_combination}')

        # Check which nodes are NOT covered by the new combination
        uncovered_nodes = list(set(range(len(valid_subsets_dict))) - set(new_combination_flattened))
        if len(uncovered_nodes) == 0:
            if is_combination_valid(new_combination, len(valid_subsets_dict)):
                print(f'********found valid combination: {new_combination}********\n')
                result.append(new_combination)
                return result
        else:
            # Recursively call the function to process the next key
            for next_key in range(current_key + 1, len(valid_subsets_dict)):
                generate_combinations(valid_subsets_dict, next_key, new_combination, result)
    
    return result


# Get all valid combinations
combinations = generate_combinations(valid_subsets_dict, 0)
print(len(combinations))

********found valid combination: [[0], [1], [2], [3], [4], [5], [6], [7], [8], [9], [10]]********

********found valid combination: [[0], [1], [2], [3], [4], [5], [6], [7], [8, 9], [10]]********

********found valid combination: [[0], [1], [2], [3], [4], [5], [6, 10], [7], [8], [9]]********

********found valid combination: [[0], [1], [2], [3], [4], [5], [6, 10], [7], [8, 9]]********

********found valid combination: [[0], [1], [2], [3], [4], [5], [6, 9], [7], [8], [10]]********

********found valid combination: [[0], [1], [2], [3], [4], [5], [6, 9, 10], [7], [8]]********

********found valid combination: [[0], [1], [2], [3], [4], [5], [6, 8, 9], [7], [10]]********

********found valid combination: [[0], [1], [2], [3], [4], [5], [6, 8, 9, 10], [7]]********

********found valid combination: [[0], [1], [2], [3], [4, 8], [5], [6], [7], [9], [10]]********

********found valid combination: [[0], [1], [2], [3], [4, 8], [5], [6, 10], [7], [9]]********

********found valid combination: [[0], [

In [205]:
# def is_combination_valid(combination, n):
#     # Flatten list of combination
#     covered_tasks_list = [element for sublist in combination for element in sublist]
    
#     # Create a set of the flattened list
#     covered_tasks_set = set(covered_tasks_list)
    
#     # Check if the flattened set has exactly n elements and contains all elements from 0 to n-1
#     if len(covered_tasks_list) == n and covered_tasks_set == set(range(n)):
#         return True
#     else:
#         return False


# def generate_combinations(valid_subsets_dict, current_key=0, current_combination=None, result=None):
#     if current_combination is None:
#         current_combination = []
#     if result is None:
#         result = []

#     # Base case: if convered all tasks add current combination to the result list
#     if is_combination_valid(current_combination, len(valid_subsets_dict)):
#         print(f'********found valid combination: {current_combination}********')
#         result.append(current_combination)
#         return result

#     # Recursive case: iterate through the list of lists at the current key
#     for subset in valid_subsets_dict[current_key]:
#         # Create a new combination including the current subset
#         new_combination = current_combination + [subset]
#         new_combination_flattened = [element for sublist in new_combination for element in sublist]

#         # Check which nodes are NOT covered by the new combination. Only process these nodes next
#         uncovered_nodes = list(set(range(len(valid_subsets_dict))) - set(new_combination_flattened))
#         #print(f'new combination: {new_combination}\n uncovered nodes: {uncovered_nodes}')
#         #uncovered_nodes_valid_subsets_dict = {key: value for key, value in valid_subsets_dict.items() if key in uncovered_nodes}

#         if len(uncovered_nodes) == 0:
#             if is_combination_valid(new_combination, len(valid_subsets_dict)):
#                 print(f'********found valid combination: {new_combination}********\n')
#                 result.append(new_combination)
#                 return result
#         else:
#             # Recursively call the function to process the next key
#             for nex_key in uncovered_nodes:
#                 generate_combinations(valid_subsets_dict, nex_key, new_combination, result)
    
#     return result

In [206]:
# combinations = []
# # for initial_node in range(len(valid_subsets_dict)):
# #     combinations.extend(generate_combinations(valid_subsets_dict, initial_node))
# combinations.extend(generate_combinations(valid_subsets_dict, 0))

# print(f'number of combinations: {len(combinations)}')

In [207]:
def normalize_sublist(sublist):
    # Sort the elements within each inner list and then sort the entire sublist
    return tuple(sorted(tuple(sorted(inner)) for inner in sublist))

def unique_lists(input_list):
    seen = set()
    unique_combinations = []

    for sublist in input_list:
        normalized = normalize_sublist(sublist)
        if normalized not in seen:
            seen.add(normalized)
            unique_combinations.append(sublist)

    return unique_combinations

In [208]:
unique_combinations = unique_lists(combinations)
print(len(unique_combinations))
#unique_combinations

7074


### Generate all possible partition schemes for the set of tasks (ignoring structre of the DAG)

In [209]:
from itertools import combinations

def partitions(set_):
    if not set_:
        yield []
        return
    for i in range(1, len(set_) + 1):
        for part in combinations(set_, i):
            remaining = set(set_) - set(part)
            if not remaining:
                yield [list(part)]
            else:
                for b in partitions(list(remaining)):
                    yield [list(part)] + b

def generate_unique_partitions(numbers):
    all_partitions = set()
    for partition in partitions(numbers):
        # Create a frozenset of frozensets to make each partition hashable and order-independent
        partition_set = frozenset(frozenset(part) for part in partition)
        all_partitions.add(partition_set)
    
    # Convert the frozensets back to lists for the final output
    unique_partitions = [list(map(list, partition)) for partition in all_partitions]

    # Sort elements
    unique_partitions = sorted([sorted(x) for x in unique_partitions], key=len)
    return unique_partitions

In [210]:
# Generate list of numbers for non-"Target" tasks in occupation
tasks_list_numbers = list(range(len(valid_subsets_dict)))

# Generate all possible partitioning schemes
all_partitions = generate_unique_partitions(tasks_list_numbers)

KeyboardInterrupt: 

### Check if partition scheme is "valid" (i.e., if its non-singleton partitions are a connected graph)

In [ ]:
def is_connected(matrix):
    # Number of nodes in the matrix
    num_nodes = matrix.shape[0]
    
    # Visited array to keep track of visited nodes
    visited = np.zeros(num_nodes, dtype=bool)
    
    # Helper function to perform DFS
    def dfs(node):
        visited[node] = True
        # Visit all the neighbors of the current node
        for neighbor in range(num_nodes):
            if matrix[node, neighbor] == 1 and not visited[neighbor]:
                dfs(neighbor)
            elif matrix[neighbor, node] == 1 and not visited[neighbor]:
                dfs(neighbor)
    
    # Start DFS from the first node (node 0)
    dfs(0)
    
    # If all nodes are visited, the matrix is connected
    return np.all(visited)


def validate_partition_using_connectedness(adjacency_matrix, tasks_list):
    # Return valid if Singleton
    if len(tasks_list) == 1:
        return True
    # Check if partition forms connected graph
    else:
        # Subset original adjacency matrix
        subset_matrix = adjacency_matrix[np.ix_(tasks_list, tasks_list)]

        # check if subset matrix is a connected graph
        subset_matrix_connected = is_connected(subset_matrix)

        # return true if connected and false otherwise
        return subset_matrix_connected

In [ ]:
# Get valid partitioning schemes from all possible partitions to cut computation load
valid_partitions = []
for scheme in all_partitions:
    # Set valid partitions count to 0
    valid_partition_count = 0
    for partition in scheme:
        valid_partition = validate_partition_using_connectedness(adjacency_matrix, partition)
        if valid_partition:
            valid_partition_count += 1
    
    # If number of valid partitions within a partition scheme is equal to 
    # number of partitions in partition scheme then partition scheme is valid
    if valid_partition_count == len(scheme):
        valid_partitions.append(scheme)

# Print stats
print(f'Number of all possible partitioning schemes: {len(all_partitions)}')
print(f'Number of valid partitioning schemes given DAG structure: {len(valid_partitions)}')

# for partition in valid_partitions:
#     print(partition)

## Missing in "Smart" method

In [ ]:
def normalize_sublist(sublist):
    # Sort the elements within each inner list and then sort the entire sublist
    return tuple(sorted(tuple(sorted(inner)) for inner in sublist))

def list_difference(list1, list2):
    # Normalize both lists
    normalized_list1 = {normalize_sublist(sublist) for sublist in list1}
    normalized_list2 = {normalize_sublist(sublist) for sublist in list2}
    
    # Find the difference
    difference = normalized_list1 - normalized_list2
    
    # Convert the normalized tuples back to the original list format
    difference_list = []
    for norm_sublist in difference:
        original_sublist = [list(inner) for inner in norm_sublist]
        difference_list.append(original_sublist)
    
    return difference_list

inBruteForce_notInSmart = list_difference(valid_partitions, unique_combinations)
print(f'In Brute Force but not in Smart: {len(inBruteForce_notInSmart)}')
for case in inBruteForce_notInSmart:
    print(case)

inSmart_notInBruteForce = list_difference(unique_combinations, valid_partitions)
print(f'\nIn Smart but not in Brute Force: {len(inSmart_notInBruteForce)}')
for case in inSmart_notInBruteForce:
    print(case)